# Portugese Meals Classification: Inference

In [2]:
import gradio as gr
import tensorflow as tf
import cv2
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
import yaml


In [3]:
%cd ..

d:\GigaFolder\projects\Portuguese-Meals-Classification


In [4]:
config = yaml.safe_load(open('config.yaml', 'r'))

In [5]:
def resize(image, image_size=config['img_shape'][0]):
    return cv2.resize(image, (image_size, image_size), interpolation=cv2.INTER_AREA)

def normalize(image):
    image = image / 255
    image = image.astype(np.float32)
    return image

def preprocess(image):
    image = resize(image)
    image = normalize(image)
    return image

In [6]:
classes = ['aletria',
           'arroz_cabidela',
           'bacalhau_bras',
           'bacalhau_natas',
           'batatas_fritas',
           'bolo_chocolate',
           'cachorro',
           'caldo_verde',
           'cozido_portuguesa',
           'croissant',
           'donuts',
           'esparguete_bolonhesa',
           'feijoada',
           'francesinha',
           'gelado',
           'hamburguer',
           'jardineira',
           'nata',
           'ovo',
           'pasteis_bacalhau',
           'pizza',
           'tripas_moda_porto',
           'waffles']

classes.sort()

encoder = LabelBinarizer()
classes_oh = np.asarray(classes)
classes_oh = encoder.fit_transform(classes_oh)
del classes_oh

In [7]:
interpreter = tf.lite.Interpreter(model_path=config['tflite_path'])
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

In [18]:
def inference(image):
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    image = preprocess(image)
    image = np.expand_dims(image, axis=0)

    interpreter.set_tensor(input_details['index'], image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details['index'])

    # res = np.argmax(output)
    pred = encoder.inverse_transform(np.array(output))
    return pred[0]

In [19]:
image = cv2.imread('C:/Users/3mpti/OneDrive/Рабочий стол/Без названия.jpg')
inference(image)

'gelado'

In [12]:
interface = gr.Interface(fn=inference, inputs='image',
                         outputs='text', title='Portugese Meals Classification')

In [15]:
interface.launch(share=True, inline = False)

Running on local URL:  http://127.0.0.1:7860/
Running on public URL: https://21728.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x23fff606200>,
 'http://127.0.0.1:7860/',
 'https://21728.gradio.app')

In [16]:
interface.close()

Closing server running on port: 7860
